<a href="https://colab.research.google.com/github/andreaderuvo/rc_car_tracking/blob/main/C4_train_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports
import os
import shutil
import urllib.request
import time

In [ ]:
#configuration
PROJECT = 'rc_car_tracking'
MODEL = 'yolo'

COLAB_BASE_PROJECT = f'/content/{PROJECT}'


DRIVE_MOUNT_FOLDER = '/content/drive'
DRIVE_BASE_PROJECT = f'{DRIVE_MOUNT_FOLDER}/MyDrive/{PROJECT}'
DRIVE_DATASET_FOLDER = f'{COLAB_BASE_PROJECT}/{MODEL}_dataset'
DRIVE_TRAINED_WEIGHTS_FOLDER = f'{COLAB_BASE_PROJECT}/{MODEL}_trained_weights'
DRIVE_TRAINING_FILES_FOLDER = f'{COLAB_BASE_PROJECT}/{MODEL}_training_files'

YOLO_TRAINED_WEIGHTS_FOLDER = f'{COLAB_BASE_PROJECT}/{MODEL}_trained_weights_{round(time.time() * 1000)}'

DARKNET_PRE_TRAINED_WEIGHTS_FOLDER = '/content/darknet_overrides'
DARKNET_PRE_TRAINED_WEIGHTS_URL = 'https://pjreddie.com/media/files/darknet53.conv.74'
DARKNET_CUSTOM_CFG_FILE_URL = 'https://raw.githubusercontent.com/andreaderuvo/rc_car_tracking/main/darknet/cfg/yolov3_train.cfg'
DARKNET_CUSTOM_MAKEFILE_URL = 'https://raw.githubusercontent.com/andreaderuvo/rc_car_tracking/main/darknet/Makefile'

DARKNET_GITHUB = 'https://github.com/AlexeyAB/darknet.git'

In [ ]:
#mount Google Drive
from google.colab import drive
drive.mount(DRIVE_MOUNT_FOLDER, force_remount=True)
!ln -s $DRIVE_BASE_PROJECT $COLAB_BASE_PROJECT

In [ ]:
#download darknet53.conv.74: initial yolo weights for training custom data
os.makedirs(DARKNET_PRE_TRAINED_WEIGHTS_FOLDER, exist_ok=True)
urllib.request.urlretrieve(DARKNET_PRE_TRAINED_WEIGHTS_URL, os.path.join(DARKNET_PRE_TRAINED_WEIGHTS_FOLDER, 'darknet53.conv.74'))

In [ ]:
#download custom yolo configuration file
urllib.request.urlretrieve(DARKNET_CUSTOM_CFG_FILE_URL, os.path.join(DARKNET_PRE_TRAINED_WEIGHTS_FOLDER, 'yolov3_train.cfg'))

In [ ]:
#download and compile darknet enabling GPU mode
shutil.rmtree('darknet', ignore_errors=True)
!git clone $DARKNET_GITHUB
%cd darknet
!rm Makefile
!wget $DARKNET_CUSTOM_MAKEFILE_URL
!make
!chmod +x darknet
%cd ..

In [ ]:
#copy training files into dataset folder
!cp -a $DRIVE_TRAINING_FILES_FOLDER/. $DRIVE_DATASET_FOLDER

with open(f'{DRIVE_DATASET_FOLDER}/labelled_data.data', 'a') as data:
  data.write('backup = ' + YOLO_TRAINED_WEIGHTS_FOLDER)

In [ ]:
#train
!darknet/darknet detector train $DRIVE_DATASET_FOLDER/labelled_data.data $DARKNET_PRE_TRAINED_WEIGHTS_FOLDER/yolov3_train.cfg $DARKNET_PRE_TRAINED_WEIGHTS_FOLDER/darknet53.conv.74 -clear -dont_show -map

In [ ]:
# Clean environment
!mv chart.png $YOLO_TRAINED_WEIGHTS_FOLDER/chart.png
!mv chart_yolov3.png $YOLO_TRAINED_WEIGHTS_FOLDER/chart_yolov3.png